# [모듈 2.1] Model Approval을 하면 EventBridge를 통해 "모델 서빙" CodePipeline 수행

이 노트북은 CodeCommit 레파지토리에 'SageMaker Pipeline config 파일(sm_pipeline_serving_config.json)'을 수정 후 push 하면, EventBridge를 통해 CodePipeline을 실행하는 노트북 입니다. 

## 참고 자료
- Amazon EventBridge 유저 가이드
   (https://docs.aws.amazon.com/eventbridge/latest/userguide/eb-what-is.html)
- Boto3 Docs : EventBridge
   (https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/events.html)



# 1. 환경 확인

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import sagemaker
import boto3
sys.path.append('./src')


# 2. 리파지토리 로컬에 클로닝
- "5_sm-serving-codepipeline" 에서 만든 서빙 용 repository name 을 사용하여, 이 repository 를 로컬에 복사 합니다.
- repository 의 repository_url 를 가져옵니다.
- 아래와 같은 형태 입니다.
```
repository_url =  "https://git-codecommit.<REGION>.amazonaws.com/v1/repos/ncf-serving-repo"
```
- "repository_url" 확인하면 이후에 로컬에 클로닝 합니다.


In [ ]:
## Codecommit Repo Name (ncf-serving-repo)
repository_name = "ncf-serving-repo"

print(repository_name)

In [ ]:
repository_url = boto3.client('codecommit').get_repository(repositoryName = repository_name).get('repositoryMetadata').get('cloneUrlHttp')
print(repository_url)

# repository_url = "<Repository URL>"

### [알림] git clone 과정에서 아래와 같은 에러 발생시 트러블 슈팅 하세요.
- [에러 메세지] 
    - CalledProcessError: Command 'b'\nrepository_url=$1\nrepository_name=$2 
-  --> 솔루션 : Kernel Restart 후 노트북 처음부터 다시 실행시켜 주세요.

In [ ]:
%%sh -s {repository_url} {repository_name}

repository_url=$1
repository_name=$2

rm -rf $repository_name

git clone $repository_url 

# 3. EventBridge Rule 생성을 위한 파라미터 세팅

In [ ]:

# code pipeline name, arn, role_arn
codepipeline_name2 = "ncf-serving-code-pipeline"
codepipeline_arn = boto3.client('codepipeline').get_pipeline(name = codepipeline_name2).get('metadata').get('pipelineArn')
eventbrideexcrole_arn = boto3.client('iam').get_role(RoleName = 'MLOps-EventBridgeRole').get('Role').get('Arn')

# EventBridge Rule Name
eventbridge_rule_name2 = "codepipelinerule2"

# EventBridge Rule의 EventPattern json 파일 경로
eventpattern_json = f'src/serving_eventpattern.json'


# 4. EventBridge Rule 정의
- EventBridge Rule 생성하기 위해서는 2 가지를 생성해야 합니다.
    - (1) EventPattern 정의 
        - "5_sm-serving-codepipeline" 에서 만든 서빙 용 repository 에 변화가 (예: Commit) 생기는 것을 감지 합니다.
    - (2) Event Target 정의
        - "5_sm-serving-codepipeline 에서 생성한 "ncf-serving-code-pipeline" 을 실행하게 합니다.

## 4.1 EventPattern 정의

In [ ]:
# Codecommit Arn
repository_arn = boto3.client('codecommit').get_repository(repositoryName = repository_name).get('repositoryMetadata').get('Arn')
print(repository_arn)

In [ ]:
eventpattern_dict = {
    "source": ["aws.codecommit"],
    "detail-type": ["CodeCommit Repository State Change"],
    "resources": [repository_arn],
    "detail": {
            "referenceType": ["branch"],
            "referenceName": ["master"]
    }
}

# print(eventpattern_dict)

In [ ]:
eventpattern_dict

#### API put_rule() 을 통해 Rule 생성

In [ ]:
import json

eventclient = boto3.client('events')

eventresponse = eventclient.put_rule(
    Name = eventbridge_rule_name2,
    EventPattern = json.dumps(eventpattern_dict)
)


In [ ]:
eventresponse

## 4.2 Event Target 설정

In [ ]:
eventresponse = eventclient.put_targets(
    Rule = eventbridge_rule_name2,
    Targets = [
        {
            'Arn' : codepipeline_arn,
            'Id' : codepipeline_name2,
            'RoleArn' : eventbrideexcrole_arn
        }
    ]
)


In [ ]:
eventresponse

# 5. 새로운 SageMaker Pipeline (Serving) 설정 파일 준비
- 로컬로 클로닝한 코드 리파지토리에서 sm_pipeline_serving_config.json 파일을 로딩 합니다. 이 파일은 아래와 같은 형태 입니다.
```
{'sm_pipeline_name': 'ncf-serving-code-pipeline-sm-pipeline',
 'endpoint_instance_type': 'ml.g4dn.xlarge',
 'ModelApprovalStatus': 'Pending',
 'bucket': 'sagemaker-ap-northeast-2-XXXXXX',
 'update_time': '2022-12-28-14-39-25'}
```
- 여기서 아래 두개의 변수의 값을 수정 합니다.
    - ModelApprovalStatus : "Approved"
    - update_time : 현재 시간 저장


## 5.1. sm_pipeline_serving_config.json 파일 로딩

In [ ]:
from common_utils import save_json, load_json

# 레포지토리의 sm_pipeline_serving_config.json 파일 위치 지정
sm_pipeline_serving_config_json_path = repository_name+"/pipelines/ncf/src/sm_pipeline_serving_config.json"

# sm_pipeline_serving_config.json 파일 내용을 Load
sm_serving_pipeline_dict = load_json(sm_pipeline_serving_config_json_path)

In [ ]:
from datetime import datetime
currentDateAndTime = datetime.now()

# 현재 시간을 Json 에 넣기 위함
currentTime = currentDateAndTime.strftime("%Y-%m-%d-%H-%M-%S")
print("The current time is", currentTime)

### ModelApprovalStatus 를 "Approved" 로 지정 (=모델 승인)

In [ ]:
# sm_pipeline_serving_config.json 파일에서 모델 승인 상태와 현재 시간만 업데이트
sm_serving_pipeline_dict['ModelApprovalStatus'] = "Approved"
sm_serving_pipeline_dict['update_time'] = currentTime

# print(sm_serving_pipeline_dict)
sm_serving_pipeline_dict

## 5.2. sm_pipeline_serving_config.json 덮어 쓰기 (Overwriting)

In [ ]:
import json
from common_utils import save_json, load_json

save_json(sm_pipeline_serving_config_json_path, sm_serving_pipeline_dict)
sm_pipeline_serving_dict = load_json(sm_pipeline_serving_config_json_path)
print (json.dumps(sm_pipeline_serving_dict, indent=2))

# 6. CodeCommit 리파지토리에 푸시 (Evnet Bridge Rule 이 트리거)
- 로컬의 클로닝한 리파지토리의  "/pipelines/ncf/src/sm_pipeline_serving_config.json" 파일 변경이 있어서, CodeCommit 에 Push 하면 생성한 Event Brige Rule 에 의하여 "ncf-serving-code-pipeline" Code Pipeline 이 실행 됩니다.

In [ ]:
%%sh -s {repository_name}
repository_name=$1
cd $repository_name
date >> pushtime.log
echo $PWD
git add .
git commit -m "Add new files"
git push origin master


# 7. 변수 저장

In [ ]:
%store eventbridge_rule_name2
%store codepipeline_name2

# 8. EventBridge Rule 및 CodePipeline 실행 확인 (콘솔 화면)

## 9.1 EventBridge Rule 생성 확인
#### 하단에 codepipeline2가 생성된 것을 확인할 수 있습니다.

![eventbridge2.png](img/eventbridge2.png)

## 9.2 codepipelinerule2 의 Event pattern 확인

![eventbridge2-eventpattern.png](img/eventbridge2-eventpattern.png)

## 9.3 codepipelinerule2의 Target 확인

![eventbridge2-target.png](img/eventbridge2-target.png)

## 9.4 EventBridge를 통한 CodePipeline 실행 확인

![codepipeline-serving-inprogress1.png](img/codepipeline-serving-inprogress1.png)

## 9.5. Endpoint 업데이트 (신규 모델로 배포)

![update_endpoint.png](img/update_endpoint.png)